In [102]:
import pandas as pd
import requests

url = 'https://circuit-trial.stg.rd.ds.sansan.com/api/cards/?offset=0&limit=2000'
r = requests.get(url)


In [103]:
response = r.json()
mst_df = pd.DataFrame(response)
mst_df

,user_id,company_id,full_name,position,company_name,address,phone_number
0,9230809757,364994639,後藤 裕太,カスタマーサポートマネージャー,合同会社森鉱業,千葉県柏市柏3-7-5,090-0143-7483
1,3479534060,1683446724,松田 太郎,次長,有限会社井上運輸,千葉県柏市柏3-7-5,070-3121-9804
2,1471907357,1683446724,伊藤 幹,技術次長,有限会社井上運輸,千葉県柏市柏3-7-5,090-1288-6691
3,7558936248,2316222488,田中 学,技術係長,清水運輸合同会社,千葉県柏市柏3-7-5,080-0493-6653
4,4665080307,2316222488,吉田 裕太,次長,清水運輸合同会社,千葉県柏市柏3-7-5,070-5370-9230
...,...,...,...,...,...,...,...
1995,5273582058,9432174675,加藤 くみ子,営業課長,株式会社鈴木保険,神奈川県相模原市南区相模大野4-6-7,070-5507-7406
1996,7332714010,769945233,太田 裕太,主任,合同会社高橋建設,神奈川県横浜市西区みなとみらい3-5-7,090-8374-4171
1997,7051851097,913706213,高橋 花子,主任,伊藤農林有限会社,神奈川県横浜市西区みなとみらい3-5-7,090-5657-4902
1998,8150889539,913706213,山口 千代,ジュニアアナリスト,伊藤農林有限会社,神奈川県横浜市西区みなとみらい3-5-7,080-2291-2489


In [104]:
mst_df["position"].unique()

array(['カスタマーサポートマネージャー', '次長', '技術次長', '技術係長', '一般社員', '係長', '主任',
       'プロダクトマネージャー', 'アナリスト', 'シニアアナリスト', 'ソフトウェアエンジニア', '営業次長', '技術課長',
       '新入社員', '技術主任', '課長', 'ジュニアアナリスト', '部長', 'システムエンジニア',
       'プロジェクトマネージャー', 'アカウントマネージャー', '営業部長', '技術部長', 'リードエンジニア', '営業主任',
       'データサイエンティスト', '営業課長', '営業係長', 'アシスタントマネージャー'], dtype=object)

In [83]:
mst_df["company_id"].nunique()
# df[df["company_id"] == "364994639"]

884

In [93]:
# dfからcompany_idとcompany_nameの辞書を作る
company_dict = mst_df[["company_id", "company_name"]].drop_duplicates().set_index("company_id")["company_name"].to_dict()
company_dict

# company_dictのvalueを指定してkeyを取得する
company_name = "株式会社Sansan"
company_id = [k for k, v in company_dict.items() if v == company_name][0]
company_id

IndexError: list index out of range

In [63]:
url =  'https://circuit-trial.stg.rd.ds.sansan.com/api/contacts/owner_users/4578252120?offset=0&limit=100'
r = requests.get(url)

In [64]:
response = r.json()
df = pd.DataFrame(response)
df.head()

,owner_user_id,owner_company_id,user_id,company_id,created_at
0,4578252120,231178500,4847288,4908596099,2021-11-19T01:20:41Z
1,4578252120,231178500,18503182,9351013269,2022-10-27T02:46:02Z
2,4578252120,231178500,379103373,2297681898,2023-01-03T06:17:59Z
3,4578252120,231178500,538139622,7492970833,2020-09-05T22:17:28Z
4,4578252120,231178500,627110756,6168105904,2023-05-11T01:44:01Z


In [81]:
owner = df["owner_user_id"][0]
user = df["user_id"][0]

owner_job = mst_df[mst_df["user_id"] == owner].position.values[0]
user_job = mst_df[mst_df["user_id"] == user].position.values[0]
print(owner_job, user_job)

主任 技術主任


In [80]:
positions = {
    7: ["部長", "技術部長", "営業部長"],
    6: ["次長", "技術次長", "営業次長", "課長", "技術課長", "営業課長"],
    5: ["係長", "技術係長", "営業係長", "主任", "技術主任", "営業主任"],
    4: ["プロダクトマネージャー", "プロジェクトマネージャー", "アカウントマネージャー", "カスタマーサポートマネージャー", "アシスタントマネージャー"],
    3: ["シニアアナリスト", "リードエンジニア"],
    2: ["一般社員", "新入社員"],
    1: ["アナリスト", "ジュニアアナリスト", "ソフトウェアエンジニア", "システムエンジニア", "データサイエンティスト"]
}

# 役職名からランクを取得する辞書
position_to_rank = {position: rank for rank, positions in positions.items() for position in positions}

def get_position_rank(position):
    return position_to_rank.get(position, None)

def compare_positions(position1, position2):
    rank1 = get_position_rank(position1)
    rank2 = get_position_rank(position2)
    
    if rank1 is None or rank2 is None:
        raise ValueError("One or both positions are invalid")
    
    return rank1 - rank2

# 使用例
position1 = "部長"
position2 = "係長"

try:
    score = compare_positions(position1, position2)
    print(f"The score between {position1} and {position2} is {score}")
except ValueError as e:
    print(e)


The score between 部長 and 係長 is 2


In [82]:
compare_positions(owner_job, user_job)

0